In [ ]:
%matplotlib inline
import numpy as np
from numpy import array

from math import ceil

from matplotlib import pylab as plt  
from matplotlib.pyplot import imshow

from pylab import imread

import skimage
from skimage import morphology, measure, draw, feature, exposure
from skimage.color import rgb2hsv, rgb2gray
from skimage.morphology import square
from skimage.feature import canny
from skimage.filters import sobel
from skimage.segmentation import watershed

import cv2

from scipy import ndimage as ndi

In [ ]:
def show_array(arr, cols=3):
    "Show a mosaic of images list and save it to jpg"
    rows = ceil(len(arr)/cols)
    plt.figure(figsize=(cols*30,rows*20))
    
    for i,img in enumerate(arr):
        ax = plt.subplot(rows, cols, i+1)
        ax.set_xticks([])
        ax.set_yticks([])
        imshow(img, cmap='gray')
#     plt.savefig("tramwaje")

def show(*args):
    """Show multiple images in a row"""
    plt.figure(figsize=(20,12))
    for i,img in enumerate(args):
        plt.subplot(1, len(args), i+1)
        imshow(img, cmap='gray')
    
def get_hsv_param(img, param = 1):
    """Return h/s/v for each pixel"""
    # param: 0 - hue, 1 - sat, 2 - value
    img_sat = []
    for row in rgb2hsv(img):
        temp = []
        for col in row:
            temp.append(col[param])
        img_sat.append(temp)
    img_sat = np.array(img_sat)
    return img_sat

def oranges(img, low_treshold, high_treshold):
    img_oranges = []
    for row in rgb2hsv(img):
        temp = []
        for col in row:
            hue = col[0]
            if (hue > low_treshold and hue < high_treshold): 
                temp.append(1)
            else: 
                temp.append(0)
        img_oranges.append(temp)
    return img_oranges

In [ ]:
tram_names = ["4_04"] #"1_02", "2_01", "5_14", ,"5_07","5_12","6_02"
tram = "4_04"

In [ ]:
imgs = []

for tram in tram_names:
    # Read image
    img = imread("dane/"+tram+".jpg")
    img_sat = get_hsv_param(img, 1)

    elevation_map = sobel(img_sat)
    # show(elevation_map)

    markers = np.zeros_like(img_sat)
    markers[img_sat < 0.4] = 1
    markers[img_sat > 0.95] = 2

    segmentation = watershed(elevation_map, markers)
    segmentation = ndi.binary_fill_holes(segmentation - 1)

    label_objects, nb_labels = ndi.label(segmentation)

    sizes = np.bincount(label_objects.ravel())
    sizes
    mask_sizes = sizes > 80
    mask_sizes[0] = 0
    cleaned = mask_sizes[label_objects]
    print(cleaned)
    
#     contours, hierarchy = cv2.findContours(np.array(cleaned).astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

#     cv2.drawContours(img, contours, -1, (0,255,0), 3)

    show(img, cleaned)

    # contour_info = [(c, cv2.contourArea(c),) for c in cv2.findContours(edges, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)[1]]
    # contour_info
    imgs.append(cleaned)
show_array(imgs)

In [ ]:
imgs = []

# for tram in tram_names:
img = imread("dane/"+tram+".jpg")

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
blurred = cv2.GaussianBlur(gray, (7,7), 0)
edged = cv2.Canny(blurred, 50, 200, 255)

# detect circles in the image
output = img.copy()
circles = cv2.HoughCircles(edged, cv2.HOUGH_GRADIENT, 1.2, 20, maxRadius=100)
# ensure at least some circles were found
if circles is not None:
    # convert the (x, y) coordinates and radius of the circles to integers
    circles = np.round(circles[0, :]).astype("int")
    # loop over the (x, y) coordinates and radius of the circles
    for (x, y, r) in circles:
        # draw the circle in the output image, then draw a rectangle
        # corresponding to the center of the circle
        cv2.circle(output, (x, y), r, (0, 255, 0), 1)
        cv2.rectangle(output, (x - 5, y - 5), (x + 5, y + 5), (0, 128, 255), -1)
    # show the output image
show(img,edged, output)


# contours, hierarchy = cv2.findContours(edged, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# cv2.drawContours(img, contours, -1, (0,255,0), 3)
# # cv2.drawContours(img_contoured, contours, -1, (255,0,0),3)

# contour_list = []
# for contour in contours:
#     area = cv2.contourArea(contour)
#     if (area <100 and area > 40):
#         contour_list.append(contour)
# cv2.drawContours(img, contour_list, -1, (255,0,0), 3)


# circles

# for contour in contours:
#     approx = cv2.approxPolyDP(contour,0.01*cv2.arcLength(contour,True),True)
#     area = cv2.contourArea(contour)
#     if ((len(approx) > 8) & (area > 30) ):
#         contour_list.append(contour)

# show(img)
# contours
#     imgs.append(gray)
#     imgs.append(edged)
# show_array(imgs)

In [ ]:
hist, hist_centers = exposure.histogram(img_sat)
plt.plot(hist_centers, hist)

# ideas:
1. więcej preprocessingu (jakieś podbijanie kontrastu, rozmycie, ustalenie jasności...)
1. operowanie jednak na skali szarości?
1. łączenie rozpoznanych obiektów ze skali szarości i z nasycenia
1. jakoś inaczej dobrać parametry przed canny?   
    - Inne rozmycie? 
    - cv2.bilateralFilter(rawImage, 5, 175, 175) zamiast gaussian?

1. zmniejszenie rozdzielczości? (Czy to ma jakieś znaczenie)
1. wykryć koła, sprawdzić ich wielkość, wyciąć i tam szukać lizczb tym algorytmem, który mam
    - może być wiele kół, może nie wykryć poprawnie koła
1. Jak wykryć te cholerne koła?


Raczej nie:
1. wykryć największy obiekt (prawdopodobnie to będzie tramwaj) i szukać na górze od niego
    - największy może nie być tramwaj, może wykrywać tramwaj w częściach
1. ograniczyć szukanie tylko do górnej połowy zdjęcia, obciąć boki
    - jest ryzyko, że tam właśnie będzie numer, bo zdjęcie będzie słabe



- transformata hougha - raczej nieprzydatna
- momenty hu - do rozpoznawania koła?
- cv2.find_contorurs
